In [5]:
# Initial imports
import numpy as np
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [6]:
# Load the crypto_data.csv dataset.

file_path = "Resources/Formula1_maindata.csv"
formula1_df = pd.read_csv(file_path)
print(formula1_df.shape)
formula1_df.head(10)

(1619, 28)


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,status,driverRef,constructorRef,circuitId,name,year,circuitRef,rank_yrs_exp_driver,rank_wins_driver,rank_quali_wins_driver
0,23783,989,1,131,44,1,2,2,2,18.0,...,Finished,hamilton,mercedes,1,Australian Grand Prix,2018,albert_park,3.0,1.0,1.0
1,23784,989,8,6,7,2,3,3,3,15.0,...,Finished,raikkonen,ferrari,1,Australian Grand Prix,2018,albert_park,1.0,4.0,4.0
2,23782,989,20,6,5,3,1,1,1,25.0,...,Finished,vettel,ferrari,1,Australian Grand Prix,2018,albert_park,3.0,2.0,2.0
3,23787,989,830,9,33,4,6,6,6,8.0,...,Finished,max_verstappen,red_bull,1,Australian Grand Prix,2018,albert_park,10.0,5.0,6.0
4,23798,989,825,210,20,5,\N,R,17,0.0,...,Wheel,kevin_magnussen,haas,1,Australian Grand Prix,2018,albert_park,12.0,13.0,13.0
5,23797,989,154,210,8,6,\N,R,16,0.0,...,Wheel,grosjean,haas,1,Australian Grand Prix,2018,albert_park,7.0,13.0,13.0
6,23788,989,807,4,27,7,7,7,7,6.0,...,Finished,hulkenberg,renault,1,Australian Grand Prix,2018,albert_park,7.0,13.0,9.0
7,23785,989,817,9,3,8,4,4,4,12.0,...,Finished,ricciardo,red_bull,1,Australian Grand Prix,2018,albert_park,5.0,7.0,8.0
8,23791,989,832,4,55,9,10,10,10,1.0,...,Finished,sainz,renault,1,Australian Grand Prix,2018,albert_park,10.0,13.0,13.0
9,23786,989,4,1,14,10,5,5,5,10.0,...,Finished,alonso,mclaren,1,Australian Grand Prix,2018,albert_park,1.0,3.0,3.0


In [12]:
# Create a new DataFrame that holds only the cryptocurrencies names.

formula1_df["win"] = np.where(formula1_df["positionOrder"] == 1, 1, 0)
raceswondf = formula1_df[["resultId","rank_yrs_exp_driver","rank_wins_driver","rank_quali_wins_driver","win"]].copy()
raceswondf.head()

,resultId,rank_yrs_exp_driver,rank_wins_driver,rank_quali_wins_driver,win
0,23783,3.0,1.0,1.0,0
1,23784,1.0,4.0,4.0,0
2,23782,3.0,2.0,2.0,1
3,23787,10.0,5.0,6.0,0
4,23798,12.0,13.0,13.0,0


In [28]:
# Define features set
X = raceswondf.copy()
X = X.drop(["win","resultId"], axis=1)
X.head()

,rank_yrs_exp_driver,rank_wins_driver,rank_quali_wins_driver
0,3.0,1.0,1.0
1,1.0,4.0,4.0
2,3.0,2.0,2.0
3,10.0,5.0,6.0
4,12.0,13.0,13.0


In [19]:
# Define target vector
y = raceswondf["win"].values

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)

# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
from sklearn.ensemble import GradientBoostingClassifier

# Create a classifier object
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=learning_rate,
                                            max_features=3,
                                            max_depth=3,
                                            random_state=0)

    # Fit the model
    classifier.fit(X_train_scaled, y_train)
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train_scaled,
            y_train)))
    print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test_scaled,
            y_test)))
    print()

Learning rate:  0.05
Accuracy score (training): 0.950
Accuracy score (validation): 0.951

Learning rate:  0.1
Accuracy score (training): 0.950
Accuracy score (validation): 0.951

Learning rate:  0.25
Accuracy score (training): 0.951
Accuracy score (validation): 0.951

Learning rate:  0.5
Accuracy score (training): 0.951
Accuracy score (validation): 0.951

Learning rate:  0.75
Accuracy score (training): 0.951
Accuracy score (validation): 0.951

Learning rate:  1
Accuracy score (training): 0.950
Accuracy score (validation): 0.951



In [24]:
# Choose a learning rate and create classifier
classifier = GradientBoostingClassifier(n_estimators=20,
                                        learning_rate=0.25,
                                        max_features=3,
                                        max_depth=3,
                                        random_state=0)

# Fit the model
classifier.fit(X_train_scaled, y_train)

# Make Prediction
predictions = classifier.predict(X_test_scaled)
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).head(20)

,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,1,1
5,0,0
6,1,0
7,0,0
8,0,0
9,0,1


In [25]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.9506172839506173


In [29]:
# Generate the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying results
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,373,12
Actual 1,8,12


In [30]:
# Generate classification report
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

           0       0.98      0.97      0.97       385
           1       0.50      0.60      0.55        20

    accuracy                           0.95       405
   macro avg       0.74      0.78      0.76       405
weighted avg       0.96      0.95      0.95       405

